In [4]:
!pip install pandas

   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/9.7 MB 6.2 MB/s eta 0:00:02
   ----------- ---------------------------- 2.9/9.7 MB 7.8 MB/s eta 0:00:01
   ------------------- -------------------- 4.7/9.7 MB 8.1 MB/s eta 0:00:01
   ---------------------------- ----------- 6.8/9.7 MB 8.6 MB/s eta 0:00:01
   ---------------------------------- ----- 8.4/9.7 MB 8.4 MB/s eta 0:00:01
   ---------------------------------------- 9.7/9.7 MB 8.0 MB/s  0:00:01
   ---------------------------------------- 0.0/12.3 MB ? eta -:--:--
   ----- ---------------------------------- 1.6/12.3 MB 8.7 MB/s eta 0:00:02
   --------- ------------------------------ 2.9/12.3 MB 7.4 MB/s eta 0:00:02
   --------------- ------------------------ 4.7/12.3 MB 7.9 MB/s eta 0:00:01
   --------------------- ------------------ 6.6/12.3 MB 7.9 MB/s eta 0:00:01
   --------------------------- ------------ 8.4/12.3 MB 8.2 MB/s eta 0:00:01
   --------------------


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd

In [6]:
d1 = pd.read_csv("lowerGerUni.csv")

In [7]:
d1

,Unnamed: 0,university,universityLabel,lat,lon,PLZ
0,0,http://www.wikidata.org/entity/Q1453964,Freie Hochschule Stuttgart,48.782200,9.192280,70188
1,1,http://www.wikidata.org/entity/Q1453965,Freie Hochschule für Grafik-Design & Bildende ...,47.992119,7.835028,79115
2,2,http://www.wikidata.org/entity/Q1667283,Internationale Hochschule Liebenzell,48.772900,8.727730,75378
3,3,http://www.wikidata.org/entity/Q1667803,Internationales Priesterseminar Herz Jesu,48.837260,12.211090,84069
4,4,http://www.wikidata.org/entity/Q1736343,Katholische Hochschule für Kirchenmusik Rotten...,48.486700,8.941670,72108
...,...,...,...,...,...,...
113,113,http://www.wikidata.org/entity/Q15852343,VWA Hochschule,48.792308,9.183608,70191
114,114,http://www.wikidata.org/entity/Q7968367,War Academy,48.147231,11.550961,80335
115,115,http://www.wikidata.org/entity/Q1622145,Weihenstephan-Triesdorf University of Applied ...,48.395708,11.730411,85354
116,116,http://www.wikidata.org/entity/Q2576313,Wilhelmsstift,48.521200,9.055150,72070


In [14]:
from __future__ import annotations

from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Optional, Iterable, Tuple


# GeoNames postal code file format (tab-separated) columns:
# 0 country_code
# 1 postal_code
# 2 place_name
# 3 admin_name1  (Bundesland)
# 4 admin_code1
# 5 admin_name2  (Regierungsbezirk / Landkreis level, varies)
# 6 admin_code2
# 7 admin_name3
# 8 admin_code3
# 9 latitude
# 10 longitude
# 11 accuracy
#
# Ref: 🧾 https://www.geonames.org/export/zip/  (see "postalCodes" readme)


@dataclass(frozen=True)
class GeoNamesPostalRecord:
    country_code: str
    postal_code: str
    place_name: str
    state_name: str
    county_name: str
    latitude: Optional[float]
    longitude: Optional[float]
    accuracy: Optional[int]


def _to_float(s: str) -> Optional[float]:
    s = s.strip()
    if not s:
        return None
    try:
        return float(s)
    except ValueError:
        return None


def _to_int(s: str) -> Optional[int]:
    s = s.strip()
    if not s:
        return None
    try:
        return int(s)
    except ValueError:
        return None


def normalize_de_plz(plz: str) -> str:
    """
    Normalize German PLZ to 5 digits (string).
    Examples: "10115" -> "10115", " 911 " -> "00911"
    """
    plz = "".join(ch for ch in str(plz).strip() if ch.isdigit())
    return plz.zfill(5)


def read_geonames_postal_txt(path: str | Path, country_code: str = "DE") -> List[GeoNamesPostalRecord]:
    """
    Read a GeoNames postal-code TXT (tab-separated) into a list of records.
    Filters by country_code (default "DE").
    """
    path = Path(path)
    records: List[GeoNamesPostalRecord] = []

    with path.open("r", encoding="utf-8", errors="replace") as f:
        for line_no, line in enumerate(f, start=1):
            line = line.rstrip("\n")
            if not line or line.startswith("#"):
                continue

            parts = line.split("\t")
            if len(parts) < 12:
                # Skip malformed lines
                continue

            cc = parts[0].strip()
            if cc != country_code:
                continue

            postal_code = parts[1].strip()
            place_name = parts[2].strip()
            admin_name1 = parts[3].strip()   # state
            admin_name2 = parts[5].strip()   # county/region (varies)

            lat = _to_float(parts[9])
            lon = _to_float(parts[10])
            acc = _to_int(parts[11])

            records.append(
                GeoNamesPostalRecord(
                    country_code=cc,
                    postal_code=postal_code,
                    place_name=place_name,
                    state_name=admin_name1,
                    county_name=admin_name2,
                    latitude=lat,
                    longitude=lon,
                    accuracy=acc,
                )
            )

    return records


def build_plz_index(records: Iterable[GeoNamesPostalRecord]) -> Dict[str, List[GeoNamesPostalRecord]]:
    """
    Build an index: PLZ -> list of matching place records.
    Note: A single PLZ can map to multiple place names.
    """
    index: Dict[str, List[GeoNamesPostalRecord]] = {}
    for r in records:
        plz = normalize_de_plz(r.postal_code)
        index.setdefault(plz, []).append(r)
    return index


def city_for_plz(
    plz: str,
    index: Dict[str, List[GeoNamesPostalRecord]],
    *,
    mode: str = "first",
) -> Optional[str] | List[str]:
    """
    Return place/city name(s) for a single PLZ.

    mode:
      - "first": return one place_name (deterministic: best accuracy, then alphabetic)
      - "all": return all unique place names (sorted)
    """
    key = normalize_de_plz(plz)
    matches = index.get(key, [])
    if not matches:
        return None

    if mode == "all":
        return sorted({m.place_name for m in matches})

    if mode != "first":
        raise ValueError("mode must be 'first' or 'all'")

    # Deterministic pick: highest accuracy (if present), then place_name
    def sort_key(r: GeoNamesPostalRecord) -> Tuple[int, str]:
        acc = r.accuracy if r.accuracy is not None else -1
        return (acc, r.place_name, r.state_name)

    best = sorted(matches, key=sort_key, reverse=True)[0]
    return (best.place_name, best.state_name)


# ---------- Example usage ----------
if __name__ == "__main__":
    records = read_geonames_postal_txt("Utils/DE.txt", country_code="DE")
    plz_index = build_plz_index(records)

    print(city_for_plz("10115", plz_index))               # single (default)
    print(city_for_plz("10115", plz_index, mode="all"))   # list of all place names


('Berlin', 'Berlin')
['Berlin']


In [11]:
d1["City"] = d1["PLZ"].apply(lambda x: city_for_plz(x, plz_index))

In [15]:
d1[["City", "State"]] = d1["PLZ"].apply(lambda x: city_for_plz(x, plz_index)).apply(pd.Series)

In [18]:
!pip install requests

  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached charset_normalizer-3.4.4-cp313-cp313-win_amd64.whl.metadata (38 kB)
  Using cached urllib3-2.6.3-py3-none-any.whl.metadata (6.9 kB)
  Using cached certifi-2026.1.4-py3-none-any.whl.metadata (2.5 kB)
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
Using cached charset_normalizer-3.4.4-cp313-cp313-win_amd64.whl (107 kB)
Using cached urllib3-2.6.3-py3-none-any.whl (131 kB)
Using cached certifi-2026.1.4-py3-none-any.whl (152 kB)

   ---------------------------------------- 0/4 [urllib3]
   -------------------- ------------------- 2/4 [certifi]
   ------------------------------ --------- 3/4 [requests]
   ---------------------------------------- 4/4 [requests]




[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from scraper_collector_thinkimmo import ThinkImmoClient, ThinkImmoQuery, ThinkImmoCollector


In [ ]:
client = ThinkImmoClient(ttl_seconds=60)
base_query = ThinkImmoQuery(
        size=20,
        offset=0,
        type="APARTMENTBUY",
        sort_by="publishDate,desc",
        # geo fields will be overridden per city
        geo_search_query="",
        geo_search_type="town",
        region="",
    )

# 5 German cities (regions are Bundesländer)
cities = [
    CitySpec("München", "city", "Bayern"),
    CitySpec("Berlin", "city", "Berlin"),
    CitySpec("Hamburg", "city", "Hamburg"),
    CitySpec("Köln", "city", "Nordrhein-Westfalen"),
    CitySpec("Frankfurt am Main", "city", "Hessen"),
]

# If you need custom headers, set them here
custom_headers = {}

collector = ThinkImmoCollector(
    client,
    headers=custom_headers,
    items_key="content",  # <-- change after you inspect a sample response
)

dataset = collector.collect_many(
    base_query,
    cities,
    max_pages_per_city=5,  # keep small while testing
)